In [1]:
%pwd

'd:\\Projects\\Kidney-Disease-Classification-with-Deep-Learning\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Projects\\Kidney-Disease-Classification-with-Deep-Learning'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [5]:
from kidney_disease_prediction.constants import *
from kidney_disease_prediction.utils.common import read_yaml, create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        base_model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        # Add resizing layer INSIDE the function
        inputs = tf.keras.Input(shape=(None, None, 3))
        x = tf.keras.layers.Resizing(
            self.config.params_image_size[0],
            self.config.params_image_size[1]
        )(inputs)
        x = base_model(x)

        self.model = tf.keras.Model(inputs, x)

        # 🔥 compile the model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]
        )

        print("Base model loaded, resized, and compiled.")

    # -----------------------------------------------------
    # Data Pipeline
    # -----------------------------------------------------
    def train_valid_generator(self):

        IMG_SIZE = tuple(self.config.params_image_size[:-1])
        BATCH = self.config.params_batch_size

        train_ds = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.20,
            subset="training",
            seed=42,
            image_size=IMG_SIZE,
            batch_size=BATCH
        )

        valid_ds = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.20,
            subset="validation",
            seed=42,
            image_size=IMG_SIZE,
            batch_size=BATCH
        )

        AUTOTUNE = tf.data.AUTOTUNE
        train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
        valid_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)

        # Optional augmentation
        if self.config.params_is_augmentation:
            data_augmentation = tf.keras.Sequential([
                tf.keras.layers.RandomFlip("horizontal"),
                tf.keras.layers.RandomRotation(0.1),
                tf.keras.layers.RandomZoom(0.1)
            ])

            train_ds = train_ds.map(
                lambda x, y: (data_augmentation(x, training=True), y),
                num_parallel_calls=AUTOTUNE
            )

        self.train_dataset = train_ds
        self.valid_dataset = valid_ds

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        print(f"Model saved at: {path}")

    def train(self):
        self.model.fit(
            self.train_dataset,
            epochs=self.config.params_epochs,
            validation_data=self.valid_dataset
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()

    training = Training(config=training_config)

    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e


[2025-12-07 19:41:37,122: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-07 19:41:37,124: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-07 19:41:37,126: INFO: common: created directory at: artifacts]
[2025-12-07 19:41:37,129: INFO: common: created directory at: artifacts\training]
[2025-12-07 19:41:37,429: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Base model loaded, resized, and compiled.
Found 465 files belonging to 2 classes.
Using 372 files for training.
Found 465 files belonging to 2 classes.
Using 93 files for validation.


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 2)